In [1]:
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import joblib
import pickle
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras import Sequential,layers, losses, optimizers
from tensorflow.keras.layers import Dense
import tensorflow as tf
import time

In [2]:
df_train = pd.read_csv("all_group_train_normalized.csv")

In [3]:
df_train['Label'].value_counts()

Benign          53432
DoS              8906
Web Attack       8906
Infiltration     8905
Port Scan        8905
DDoS             8905
Brute Force      8905
Name: Label, dtype: int64

In [4]:
df_test = pd.read_csv("all_group_test_normalized.csv")

In [5]:
infiltration_attack = df_test[df_test['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(infiltration_attack.shape)
portscan_attack = df_test[df_test['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(portscan_attack.shape)
ddos_attack = df_test[df_test['Label'] == 'DDoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(ddos_attack.shape)
bruteforce_attack = df_test[df_test['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(bruteforce_attack.shape)
dos_attack = df_test[df_test['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(dos_attack.shape)
web_attack = df_test[df_test['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(web_attack.shape)
benign_data = df_test[df_test['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:200]
print(benign_data.shape)

(3817, 200)
(3817, 200)
(3817, 200)
(3817, 200)
(3816, 200)
(3816, 200)
(22900, 200)


In [6]:
infiltration_attack

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_125,tcp_segment_data_byte_126,tcp_segment_data_byte_127,tcp_segment_data_byte_128,tcp_segment_data_byte_129,tcp_segment_data_byte_130,tcp_segment_data_byte_131,tcp_segment_data_byte_132,tcp_segment_data_byte_133,tcp_segment_data_byte_134
5,0.007843,0.109804,0.054902,0.980392,0.25098,0.0,0.501961,0.674510,0.223529,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
14,0.003922,0.717647,0.286275,0.760784,0.25098,0.0,0.501961,0.443137,0.839216,0.929412,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
16,0.003922,0.835294,0.172549,0.192157,0.25098,0.0,0.501961,0.560784,0.286275,0.196078,...,0.050980,0.039216,0.262745,0.227451,0.360784,0.243137,0.431373,0.427451,0.380392,0.439216
19,0.007843,0.109804,0.054902,0.494118,0.25098,0.0,0.501961,0.674510,0.709804,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
28,0.007843,0.109804,0.145098,0.443137,0.25098,0.0,0.501961,0.584314,0.760784,0.929412,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45759,0.003922,0.133333,0.215686,0.541176,0.25098,0.0,0.501961,0.517647,0.639216,0.196078,...,0.050980,0.039216,0.262745,0.227451,0.360784,0.243137,0.431373,0.427451,0.380392,0.439216
45761,0.007843,0.686275,0.180392,0.145098,0.25098,0.0,0.501961,0.549020,0.482353,0.196078,...,0.152941,0.431373,0.427451,0.380392,0.439216,0.152941,0.125490,0.411765,0.450980,0.125490
45768,0.000000,0.270588,0.086275,0.329412,0.25098,0.0,0.501961,0.650980,0.713725,0.196078,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
45781,0.011765,0.168627,0.462745,0.356863,0.25098,0.0,0.501961,0.262745,0.788235,0.192157,...,0.450980,0.298039,0.125490,0.176471,0.431373,0.125490,0.192157,0.223529,0.196078,0.180392


In [7]:
df_test_2018 = pd.read_csv("all_group_train_normalized_2018.csv")

In [8]:
df_test_2018 = df_test_2018.sample(frac = 1).reset_index(drop = True)

In [9]:
df_test_2018

,ip_header_byte_2,ip_header_byte_3,ip_header_byte_4,ip_header_byte_5,ip_header_byte_6,ip_header_byte_7,ip_header_byte_8,ip_header_byte_10,ip_header_byte_11,tcp_header_byte_4,...,tcp_segment_data_byte_1452,tcp_segment_data_byte_1453,tcp_segment_data_byte_1454,tcp_segment_data_byte_1455,tcp_segment_data_byte_1456,tcp_segment_data_byte_1457,tcp_segment_data_byte_1458,tcp_segment_data_byte_1459,Label,Label_binary
0,0.000000,0.650980,0.262745,0.298039,0.25098,0.0,0.250980,0.941176,0.262745,0.529412,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
1,0.000000,0.203922,0.090196,0.878431,0.25098,0.0,0.250980,0.639216,0.921569,0.615686,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
2,0.000000,0.156863,0.003922,0.537255,0.25098,0.0,0.498039,0.509804,0.960784,0.168627,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Web Attack,Malicious
3,0.000000,0.337255,0.384314,0.313725,0.25098,0.0,0.501961,0.176471,0.552941,0.243137,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
4,0.000000,0.352941,0.094118,0.568627,0.25098,0.0,0.427451,0.552941,0.286275,0.156863,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Infiltration,Malicious
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291153,0.000000,0.156863,0.007843,0.325490,0.25098,0.0,0.501961,0.109804,0.160784,0.356863,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
291154,0.000000,0.156863,0.435294,0.141176,0.25098,0.0,0.501961,0.686275,0.345098,0.858824,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
291155,0.000000,0.203922,0.090196,0.803922,0.25098,0.0,0.501961,0.125490,0.768627,0.184314,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Benign,Benign
291156,0.000000,0.156863,0.149020,0.603922,0.25098,0.0,0.498039,0.921569,0.203922,0.780392,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ddos,Malicious


In [10]:
X_train_2018 = df_test_2018.iloc[:,0:1525]
Y_train_2018 = df_test_2018.iloc[:,1526:1527]

In [11]:
X_train_2017 = df_test.iloc[:,0:1525]
Y_train_2017 = df_test.iloc[:,1526:1527]

In [12]:
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2017.loc[Y_train_2017['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [13]:
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Benign', 'Label_binary'] = 0
Y_train_2018.loc[Y_train_2018['Label_binary'] == 'Malicious', 'Label_binary'] = 1

In [14]:
X_train_total = pd.concat([X_train_2017,X_train_2018])
Y_train_total = pd.concat([Y_train_2017,Y_train_2018])

In [15]:
X_train_ = X_train_total.values.astype('float32')
Y_train_ = Y_train_total.values.astype('float32')

In [16]:
infiltration_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Infiltration'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(infiltration_attack_2018.shape)
portscan_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Port Scan'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(portscan_attack_2018.shape)
ddos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'ddos'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(ddos_attack_2018.shape)
bruteforce_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Brute Force'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(bruteforce_attack_2018.shape)
dos_attack_2018 = df_test_2018[df_test_2018['Label'] == 'DoS'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(dos_attack_2018.shape)
web_attack_2018 = df_test_2018[df_test_2018['Label'] == 'Web Attack'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(web_attack_2018.shape)
benign_data_2018 = df_test_2018[df_test_2018['Label'] == 'Benign'].drop(['Label_binary','Label'], axis=1).iloc[:,0:500]
print(benign_data_2018.shape)

(29116, 500)
(0, 500)
(29115, 500)
(29116, 500)
(29116, 500)
(29116, 500)
(145579, 500)


In [17]:
benign_data_total = pd.concat([benign_data, benign_data_2018])

In [18]:
# Y_train.loc[Y_train['Label_binary'] == 'Benign', 'Label_binary'] = 0

In [19]:
stage3_AE = tf.keras.models.load_model('Stage3_AE.h5')

In [20]:
def recon_metrices(data, reconstructed_data):

    maes = np.absolute(data.values - reconstructed_data)
    
    for j in range(len(maes)):
        m = maes[j]
        d = data.values[j]
        r = reconstructed_data[j]
        for el in range(len(m)):
            if r[el] == 0 or d[el] == 0:
                m[el] = 0
 
            
    mae_byte_list = []
    ip_mae_list = []
    tcp_header_mae_list = []
    tcp_options_mae_list = []
    tcp_segment_mae_list = []
    i=0
    for mae in maes:
#         print(np.count_nonzero(reconstructed_data[i]))
#         print(np.count_nonzero(data.values[i]))
#         print(np.max(np.count_nonzero(reconstructed_data[i]),np.count_nonzero(data.values[i])))
        mae_byte = sum(mae) / np.count_nonzero(mae)
        mae_byte_list.append(mae_byte)
        i += 1
    return mae_byte_list

In [21]:
inf = stage3_AE.predict(infiltration_attack_2018)

910/910 [==============================] - 5s 5ms/step


In [22]:
mae_byte_list = recon_metrices(infiltration_attack_2018,inf)

In [23]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.09])
print(count_99 / len(mae_byte_list))

0.8673238082154142


In [25]:
stage3_AE.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               250500    
                                                                 
 dense_1 (Dense)             (None, 400)               200400    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 300)               120300    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 250)               75250     
                                                                 
 dense_4 (Dense)             (None, 125)               3

In [ ]:
### Retraining with CICIDS 2018 data

In [33]:
mod_layers = [5,6,8,9]

In [34]:
layer5_ind = [h for h in range(0,250,3)]
layer6_ind = [h for h in range(0,125,3)]
layer8_ind = [h for h in range(0,125,3)]
layer9_ind = [h for h in range(0,250,3)]

In [35]:
retrained_stage3 = tf.keras.Sequential()

In [36]:
for i, layer in enumerate(stage3_AE.layers):
    if i not in mod_layers:
        layer.trainable = False
        retrained_stage3.add(layer)
    else:
        retrained_stage3.add(layer)
        if i == 5:
            for ind in range(0,250):
                if ind in layer5_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
        if i == 6:
            for ind in range(0,125):
                if ind in layer6_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
                    
        if i == 8:
            for ind in range(0,62):
                if ind in layer8_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False
        
        if i == 9:
            for ind in range(0,125):
                if ind in layer9_ind:
                    retrained_stage3.layers[-1].kernel[ind]._trainable = False

In [37]:
retrained_stage3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 500)               250500    
                                                                 
 dense_1 (Dense)             (None, 400)               200400    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 300)               120300    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 300)               0         
                                                                 
 dense_3 (Dense)             (None, 250)               75250     
                                                                 
 dense_4 (Dense)             (None, 125)              

In [38]:
df_adv = pd.read_csv("adv_examples_all_allDNN_bigger.csv")
df_adv_syn = pd.read_csv("Adv_all_DNN_incremental.csv")

In [39]:
df_adv_syn= df_adv_syn.loc[:, ~df_adv_syn.columns.str.contains('^Unnamed')]

In [40]:
adv_train = df_adv.iloc[:,:500]

In [41]:
adv_syn = df_adv_syn.iloc[:,:500]

In [42]:
adv_total = pd.concat([adv_train,adv_syn])

In [43]:
start_time = time.time()
retrained_stage3.compile(optimizer='adam', loss='mae')
retrained_stage3.fit(adv_total , adv_total , batch_size=256, epochs=500, shuffle=True)
end_time = time.time()

Epoch 1/500
66/66 [==============================] - 2s 14ms/step - loss: 0.0084
Epoch 2/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0073
Epoch 3/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0070
Epoch 4/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0068
Epoch 5/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0066
Epoch 6/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0065
Epoch 7/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0064
Epoch 8/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0063
Epoch 9/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0062
Epoch 10/500
66/66 [==============================] - 1s 12ms/step - loss: 0.0061
Epoch 11/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0060
Epoch 12/500
66/66 [==============================] - 1s 12ms/step - loss: 0.0059
Epoch 13/500
66/66 [=====

66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 102/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 103/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 104/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 105/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 106/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 107/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 108/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 109/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 110/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 111/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0050
Epoch 112/500
66/66 [==============================] - 1s 13ms/step - loss: 0.0051
Epoch 113/500
66/6

66/66 [==============================] - 1s 18ms/step - loss: 0.0047
Epoch 298/500
66/66 [==============================] - 1s 19ms/step - loss: 0.0047
Epoch 299/500
66/66 [==============================] - 1s 17ms/step - loss: 0.0047
Epoch 300/500
66/66 [==============================] - 1s 17ms/step - loss: 0.0047
Epoch 301/500
66/66 [==============================] - 1s 16ms/step - loss: 0.0047
Epoch 302/500
66/66 [==============================] - 1s 17ms/step - loss: 0.0047
Epoch 303/500
66/66 [==============================] - 1s 16ms/step - loss: 0.0047
Epoch 304/500
66/66 [==============================] - 1s 16ms/step - loss: 0.0047
Epoch 305/500
66/66 [==============================] - 1s 17ms/step - loss: 0.0047
Epoch 306/500
66/66 [==============================] - 2s 24ms/step - loss: 0.0047
Epoch 307/500
66/66 [==============================] - 1s 21ms/step - loss: 0.0047
Epoch 308/500
66/66 [==============================] - 2s 24ms/step - loss: 0.0047
Epoch 309/500
66/6

66/66 [==============================] - 1s 18ms/step - loss: 0.0046
Epoch 494/500
66/66 [==============================] - 1s 16ms/step - loss: 0.0046
Epoch 495/500
66/66 [==============================] - 1s 15ms/step - loss: 0.0046
Epoch 496/500
66/66 [==============================] - 1s 17ms/step - loss: 0.0046
Epoch 497/500
66/66 [==============================] - 1s 15ms/step - loss: 0.0046
Epoch 498/500
66/66 [==============================] - 1s 15ms/step - loss: 0.0046
Epoch 499/500
66/66 [==============================] - 1s 14ms/step - loss: 0.0046
Epoch 500/500
66/66 [==============================] - 1s 15ms/step - loss: 0.0046


In [59]:
retrained_stage3.save('retrained_stage3_neuronfreeze.h5')

In [49]:
adv_test = retrained_stage3.predict(adv_syn)

163/163 [==============================] - 1s 3ms/step


In [46]:
time_taken = end_time - start_time
print(time_taken)

499.1160912513733


In [74]:
mae_byte_list = recon_metrices(adv_syn,adv_test)

In [75]:
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

mae_byte_avg 0.02709251992947739
mae_byte_99_percentile 0.1551133202528618
mae_byte_95_percentile 0.10022516703973731
mae_byte_90_percentile 0.0610183094324957


In [ ]:
## CICIDS2017 and CICIDS2018 Testing retrained model

In [ ]:
inf_ = retrained_stage2.predict(infiltration_attack)
port_ = retrained_stage2.predict(portscan_attack)
dos_ = retrained_stage2.predict(dos_attack)
ddos_ = retrained_stage2.predict(ddos_attack)
web_ = retrained_stage2.predict(web_attack)
brute_ = retrained_stage2.predict(bruteforce_attack)
ben_ = retrained_stage2.predict(benign_data)

In [ ]:
mae_byte_list = recon_metrices(benign_data,ben_)

In [ ]:
mae_byte_avg = sum(mae_byte_list) / len(mae_byte_list)
print('mae_byte_avg', mae_byte_avg)
mae_byte_per_95 = np.percentile(mae_byte_list, 99)
print('mae_byte_99_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 95)
print('mae_byte_95_percentile', mae_byte_per_95)
mae_byte_per_95 = np.percentile(mae_byte_list, 90)
print('mae_byte_90_percentile', mae_byte_per_95)

In [52]:
inf_2018 = retrained_stage3.predict(infiltration_attack_2018)
dos_2018 = retrained_stage3.predict(dos_attack_2018)
ddos_2018 = retrained_stage3.predict(ddos_attack_2018)
web_2018 = retrained_stage3.predict(web_attack_2018)
brute_2018 = retrained_stage3.predict(bruteforce_attack_2018)
ben_2018 = retrained_stage3.predict(benign_data_2018)

4550/4550 [==============================] - 17s 4ms/step


In [67]:
mae_byte_list = recon_metrices(infiltration_attack_2018, inf_2018)

In [68]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.10])
print(count_99 / len(mae_byte_list))

0.5168979255392224


In [71]:
df_rand = pd.read_csv('Adv_all_DNN_truerandom.csv')
df_rand = df_rand.loc[:, ~df_rand.columns.str.contains('^Unnamed')]
adv_rand = df_rand.iloc[:,:500]

In [78]:
rand_adv = retrained_stage3.predict(adv_train)
mae_byte_list = recon_metrices(adv_train, rand_adv)

362/362 [==============================] - 1s 3ms/step


In [79]:
## Classification
count_99 = len([i for i in mae_byte_list if i > 0.10])
print(count_99 / len(mae_byte_list))

0.015135789655768898


In [ ]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf_2018)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos_2018)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos_2018)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web_2018)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute_2018)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben_2018)
print(cnt_ben)
print(1-acc_ben)

In [ ]:
## CICIDS2017 Testing Retrained Model

In [ ]:
inf = retrained_stage1.predict(infiltration_attack)
dos = retrained_stage1.predict(dos_attack)
ddos = retrained_stage1.predict(ddos_attack)
web = retrained_stage1.predict(web_attack)
brute = retrained_stage1.predict(bruteforce_attack)
ben = retrained_stage1.predict(benign_data)

In [ ]:
print('Infiltration')
cnt_inf, acc_inf = accuracy_calc(inf)
print(cnt_inf)
print(acc_inf)
print('DoS')
cnt_dos, acc_dos = accuracy_calc(dos)
print(cnt_dos)
print(acc_dos)
print('DDoS')
cnt_ddos, acc_ddos = accuracy_calc(ddos)
print(cnt_ddos)
print(acc_ddos)
print('Web')
cnt_web, acc_web = accuracy_calc(web)
print(cnt_web)
print(acc_web)
print('Brute')
cnt_brute, acc_brute = accuracy_calc(brute)
print(cnt_brute)
print(acc_brute)
print('Benign')
cnt_ben, acc_ben = accuracy_calc(ben)
print(cnt_ben)
print(1-acc_ben)